# Install and Import all the necessary packages and libraries

In [1]:
pip install paretoset

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10, cifar100
import numpy as np
import time
import math
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import gc
import multiprocessing
from multiprocessing import Process, Queue
from paretoset import paretoset

# Definition of programmer's parameters (Controller's block)

In [3]:
#@markdown ##Dataset
topic_prefix = "cifar10" # @param ["cifar10", "cifar100", "mnist", "fashion_mnist"] {type:"string"}
if topic_prefix == "cifar10":
  dataset = cifar10.load_data()
elif topic_prefix == "cifar100":
  dataset = cifar100.load_data()
elif topic_prefix == "mnist":
  dataset = mnist.load_data()
elif topic_prefix == "fashion_mnist":
  dataset = fashion_mnist.load_data()

#@markdown ##Synopses-based Training Optimization Configuration
# Percentage of the dataset that will be used for training
sample_size_low = 0.05 #@param {type:"slider", min:0, max:0.99, step:0.05}
sample_size_step = 0.15 #@param {type:"number"}
sample_size_high = 1 #@param {type:"slider", min:0, max:1, step:0.05}

# Percentage of the dataset that will be used for testing
perc_test = 1 #@param {type:"number"}

# The ID of the method that will be employed during sampling
# 0: Simple reservoir sampling in our initial training dataset
# 1: Reservoir sampling in each class based on the number of samples (per class)
sampling_method_id = 1 #@param {type:"integer"}

#@markdown ##NN Architecture Configuration

total_num_of_layers = 5 #@param {type:"integer"}

set_of_layers = ['conv', 'pool', 'dense'] #, 'drop']

# Number of epochs for the training process
# One Epoch is when an ENTIRE (training) dataset is passed forward and backward through the neural network only once.
# NOTE: An epoch is comprised of one or more batches.
num_of_epochs_low = 1 #@param {type:"slider", min:1, max:30, step:1}
num_of_epochs_step = 4 #@param {type:"integer"}
num_of_epochs_high = 10 #@param {type:"slider", min:1, max:30, step:1}

# The value for the learning rate for the training process (it is inserted to the Adam optimizer)
# During the optimization, the algorithm needs to take a series of tiny steps to descend the error mountain in order to minimize the error.
# The direction of the step is determined by the gradient, while the step size is determined by the learning rate.
lr_low = 1e-3 #@param {type:"number"}
lr_high = 1e-3 #@param {type:"number"}
lr_list = np.geomspace(lr_low, lr_high, num = int(np.log10(lr_high) - np.log10(lr_low)) + 1).tolist()

# The size of the batch for the training process
# Total number of training examples present in a single batch.
# or, number of samples processed before the model is updated.
size_of_batch_low = 64 #@param {type:"integer"}
size_of_batch_step = 16 #@param {type:"integer"}
size_of_batch_high = 64 #@param {type:"integer"}

#@markdown ##Tradeoff score Configuration
# Maximum training time above which we cancel the experiment (in seconds)
# It is used in the score (accuracy--speed tradeoff) formula
theta_parameter = 10 #@param {type:"number"}

# Weight for the accuracy of the model. Max value: 0.99
# There is also the weight of training speed of the model (1 - lamda_acc)
# It is used in the score (accuracy--speed tradeoff) formula
lamda_acc = 0.5 #@param {type:"number"}


#@markdown ##Other Configuration(s)

CONV_NEURONS_CONST = 32 #@param {type:"integer"}
CONV_NEURONS_BOUND = 256 #@param {type:"integer"}
DENSE_NEURONS_CONST = 128 #@param {type:"integer"}
DENSE_NEURONS_BOUND = 32 #@param {type:"integer"}


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step


In [4]:
# Just to check that we have the correct list of learning rates
lr_list

[0.001]

# Load our image dataset

In [5]:
(train_images_all, train_labels_all), (test_images_all, test_labels_all) = dataset

# Get unique labels in our training dataset
unique_class_labels = np.unique(train_labels_all)

# Normalize pixel values in the trainning and the test datasets to be between 0 and 1
# TODO: parameter of normalization minmax scalar
train_images_all, test_images_all = train_images_all / 255.0, test_images_all / 255.0

if len(train_images_all.shape) == 3:
  # Make sure images have shape (28, 28, 1)
  train_images_all = np.expand_dims(train_images_all, -1)
  test_images_all = np.expand_dims(test_images_all, -1)
  print("x_train shape:", train_images_all.shape)
  print(train_images_all.shape[0], "train samples")
  print(test_images_all.shape[0], "test samples")

# Synopsis block of code

Optional: Check that our sampling method works appropriately

In [6]:
# A function that prints the occurence of each class in a list
def print_times_per_label(lst, labels_all):
  # Get unique labels in our training dataset
  unique_labels = np.unique(labels_all)
  for i in range(0, len(unique_labels)):
    print("Class", unique_labels[i], "has", lst.count(i), "samples in our dataset...")

**Reservoir Sampling Function**:
*Randomized algorithms for randomly choosing k samples from a list of n items, where n is either a very large or unknown number. Typically n is large enough that the list doesn’t fit into main memory.*

Algorithm implemented:


1.   Create an array reservoir[0,...,k-1] and copy first k items of stream[ ] to it.
2.   Now one by one consider all items from (k+1)th item to nth item.


> Steps

*   Generate a random number, denoted as j, from 0 to i where i is the index of the current item in stream[].
*   If j is in range 0 to k-1, replace reservoir[j] with stream[i]

In [7]:
# Select k items from a stream of items-data
import random

# A function to randomly select k items from stream[0..n-1].
def reservoir_sampling(stream, n, k):
  i = 0     # index for elements in stream[]

  # reservoir[] is the output array.
  # Initialize it with first k elements from stream[]
  reservoir = [0] * k

  for i in range(k):
    reservoir[i] = stream[i]

  # Iterate from the (k+1)th element to Nth element
  while(i < n):
    # Pick a random index from 0 to i.
    j = random.randrange(i+1)

    # If the randomly picked
    # index is smaller than k,
    # then replace the element
    # present at the index
    # with new element from stream
    if(j < k):
      reservoir[j] = stream[i]
    i+=1

  return reservoir

In [8]:
# A function that finds the size of each reservoir for every class depending on its occurence in the initial dataset
# and returns the unique labels that exist in our dataset along with the corresponding percentage
def reservoir_size_per_class(init_labels):

  # Get unique labels and their counts (how many times they appear) in our training dataset
  unique_labels, counts = np.unique(init_labels, return_counts = True)

  # Transform to list
  unique_labels_lst = unique_labels.tolist()
  counts_lst = counts.tolist()

  perc_per_class = []
  for i in range(len(unique_labels_lst)):
    perc_per_class.append(counts_lst[i]/len(init_labels))

  # print(perc_per_class)

  return perc_per_class, unique_labels_lst

"Pre-Processing Part (or Filtering)": Call the sampling mehod. Get samples from the training and the testing datasets.

In [9]:
def sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test):
  print("Percentage of filtering in our training dataset was set:")
  print(sample_size)
  if sampling_method_id == 0:
    # Simple reservoir sampling over the whole training dataset
    # Total size of the stream (or training dataset)
    n_train = len(train_images_all)

    # Number of samples that will be drawn
    k_train = int(n_train * sample_size)

    # Use the indexes of dataset in order to decide which samples will be drawn
    idx_tmp_train_list = list(range(0, n_train))

    # Find the indexes in order to construct the dataset that will be used during the training process
    idx_train = reservoir_sampling(idx_tmp_train_list, n_train, k_train)
  else:
    # Reservoir sampling in each class based on the number of samples (per class) that exist in the initial dataset
    # Find the size of each reservoir for every class depending on its occurence in the initial training dataset
    class_perc, unique_ids = reservoir_size_per_class(train_labels_all)

    # Stores the indexes (from all classes) in order to construct the dataset that will be used during the training process
    idx_train = []

    # Run for every single class the reservoir sampling seperately
    for i in range(0, len(unique_ids)):
      # Find the locations of each sample belonging to our class of interest
      tmp = np.where(train_labels_all == unique_ids[i])
      idx_of_class = tmp[0].tolist()

      # Run the reservoir sampling for the class of interest
      sampled_idx_of_class = reservoir_sampling(idx_of_class, len(idx_of_class), int(len(train_images_all) * sample_size * class_perc[i]))

      # Store the (sampled) samples from this class
      for j in range(0, len(sampled_idx_of_class)):
        idx_train.append(sampled_idx_of_class[j])

  # Store the corresponding images and labels from training dataset based on the sampled indexes
  train_images_lst = []
  for i in idx_train:
    train_images_lst.append(train_images_all[i])

  train_labels_lst = []
  for i in idx_train:
    train_labels_lst.append(train_labels_all[i])

  # Check the occurence of each class in the final training dataset
  print_times_per_label(train_labels_lst, train_labels_all)

  # Total size of the stream (or testing dataset)
  n_test = len(test_images_all)

  # Number of samples that will be drawn
  k_test = int(n_test * perc_test)

  # Use the indexes of dataset in order to decide which samples will be drawn
  idx_tmp_test_list = list(range(0, n_test))

  # Find the indexes in order to construct the dataset that will be used during the testing process
  idx_test = reservoir_sampling(idx_tmp_test_list, n_test, k_test)

  # Store the corresponding images and labels from testing dataset based on the sampled indexes
  test_images_lst = []
  for i in idx_test:
    test_images_lst.append(test_images_all[i])

  test_labels_lst = []
  for i in idx_test:
    test_labels_lst.append(test_labels_all[i])

  # Tranfsorm the lists that we stored our samples into arrays
  train_images = np.asarray(train_images_lst)
  train_labels = np.asarray(train_labels_lst)
  test_images = np.asarray(test_images_lst)
  test_labels = np.asarray(test_labels_lst)

  # Verify that the desired filtering was performed in both datasets
  print("Training dataset before sampling:")
  print(train_images_all.shape)
  print(train_labels_all.shape)
  print("Training dataset after sampling:")
  print(train_images.shape)
  print(train_labels.shape)

  print("Testing dataset before sampling:")
  print(test_images_all.shape)
  print(test_labels_all.shape)
  print("Testing dataset after sampling:")
  print(test_images.shape)
  print(test_labels.shape)

  return train_images, train_labels, test_images, test_labels

# CNN's architecture builder

In [10]:
def recreate_model(layers_lst, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):
  # Initialize a sequential model
  model = models.Sequential()

  # Define the number of neurons for conv and dense layers
  conv_tmp2 = CONV_NEURONS_CONST
  dense_tmp2 = DENSE_NEURONS_CONST

  # Recreate the model
  for count, layer in enumerate(layers_lst):
    # First layer has to be a convolutional one
    if layer == 'conv' and count == 0:
      model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu', input_shape = dataset_shape))
      conv_tmp2 = conv_tmp2 * 2
    # For the other layers
    else:
      if layer == 'conv':
        # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
        if conv_tmp2 <= CONV_NEURONS_BOUND:
          model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu'))
          conv_tmp2 = conv_tmp2 * 2
        else:
          model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
          conv_tmp2 = CONV_NEURONS_BOUND
      elif layer == 'pool':
        # Add a pool layer
        model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
      elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
      else:
        # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
        if dense_tmp2 == DENSE_NEURONS_CONST:
          model.add(layers.Flatten())
        # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
        if dense_tmp2 >= DENSE_NEURONS_BOUND:
          model.add(layers.Dense(int(dense_tmp2), activation='relu'))
          dense_tmp2 = dense_tmp2 / 2
        else:
          model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))
          dense_tmp2 = DENSE_NEURONS_BOUND

  return model, conv_tmp2, dense_tmp2

In [11]:
def my_evaluate_first_phase(q, train_images, train_labels, test_images, test_labels, epochs, lr, size_of_batch):
  # Initialize a sequential network
  model = models.Sequential()

  # Define the neurons of conv and dense layers based on user's input
  conv_tmp = CONV_NEURONS_CONST
  dense_tmp = DENSE_NEURONS_CONST

  # Add the (first) conv layer
  model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
  conv_tmp = conv_tmp * 2

  # Add manually a flatten and a dense layer in order to evaluate the network
  model.add(layers.Flatten())
  model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

  model.compile(optimizer=Adam(lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  model.summary()

  start = time.time()

  blackbox = model.fit(x=train_images,
                      y=train_labels,
                      epochs=epochs,
                      batch_size=size_of_batch
                      )
  stop = time.time()

  # Compute the training speed of this CNN architecture
  tr_time = stop - start

  # Compute the accuracy of our training model in the testing dataset
  test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

  # Compute the metric that captures the accuracy--speed tradeoff
  # Original formula
  # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

  # Linear formula
  # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

  # Multiplicative formula
  # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

  # Harmonic mean
  # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

  # Softmin
  # tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

  # Sigmoid
  # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

  # Piecewise
  # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

  # F1 like
  # alpha = 2
  # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

  # Exp decay penalty
  # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

  # inverse additive penalty
  # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

  # min-based tradeoff
  # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

  # product with complement
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

  # square root compensation
  # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

  # jacard style normalization
  # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

  # log-sigmoid dominance
  tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

  # Delete the Keras model with these hyper-parameters from memory.
  del model

  print("========================== EDW EINAI TO MULTI1111111111111111.......")
  print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
  print(f"Training time: ", tr_time)
  print(tradeOff_metric)

  q.put([test_acc, tr_time, tradeOff_metric])

In [12]:
def my_evaluate_rest_phase(q, train_images, train_labels, test_images, test_labels, layer, current_df, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):
  error_flag = -1

  # Recreate the network that consist of the best layers that we found in each of the previous steps/iterations
  model, conv_tmp, dense_tmp = recreate_model(current_df['LayerType'], dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND)

  try:
    # If the to-be-added-layer is conv
    if layer == 'conv':
      # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
      if conv_tmp <= CONV_NEURONS_BOUND:
        model.add(layers.Conv2D(int(conv_tmp), (3, 3), activation='relu'))
        conv_tmp = conv_tmp * 2
      else:
        model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
    # If the to-be-added-layer is pool
    elif layer == 'pool':
      model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    elif layer == 'none':
      print("initial iterations of greedy have a fixed/predetermined architecture")
    elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
    # If the to-be-added-layer is dense
    else:
      # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
      if not isinstance(model.layers[-1], tf.keras.layers.Dense):
        model.add(layers.Flatten())
      # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
      if dense_tmp >= DENSE_NEURONS_BOUND:
        model.add(layers.Dense(int(dense_tmp), activation='relu'))
        dense_tmp = dense_tmp / 2
      else:
        model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))


    # Check if the last layer is a Dense layer
    last_layer = model.layers[-1]

    # Check if the last layer of the network is dense
    # If it is just add the last dense layer for the classification
    # Otherwise you should first add a flatten layer
    if isinstance(last_layer, tf.keras.layers.Dense):
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
    else:
        model.add(layers.Flatten())
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

  except ValueError:
        print("No valid input...:(")
        error_flag = 1

  if error_flag == -1:
    model.compile(optimizer=Adam(current_df['LearningRate']), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.summary()

    start = time.time()

    blackbox = model.fit(x=train_images,
                        y=train_labels,
                        epochs=current_df['Epochs'],
                        batch_size=current_df['BatchSize']
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    # Original formula
    # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Softmin
    # tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 2
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    # Delete the Keras model with these hyper-parameters from memory.
    del model

    print("========================== EDW EINAI TO MULTI2222222222.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])
  else:
    q.put([0, 1000000000, 0])

In [13]:
start_program = time.time()

# Get the shape of the input dataset
dataset_shape = train_images_all.shape[1:]

# Store the best type of layer of each step
best_layers = []

# Loop for the number of layers that user indicated
for step in range(1, total_num_of_layers + 1):

  # If this is the first step/iteration
  if step == 1:

    # The first layer in this set of datasets has to be a convolutional one!
    # So our "search space" is all combinations of epochs and sampling rates based on user's input
    for sample_size in np.arange(sample_size_low, sample_size_high + 0.01, sample_size_step):
      for epochs in range(num_of_epochs_low, num_of_epochs_high + 1, num_of_epochs_step):
        for lr in lr_list:
          for size_of_batch in range(size_of_batch_low, size_of_batch_high + 1, size_of_batch_step):

            # Perform the sampling
            train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test)
            q = Queue()
            process_eval = multiprocessing.Process(target=my_evaluate_first_phase, args=(q, train_images, train_labels, test_images, test_labels, epochs, lr, size_of_batch))
            process_eval.start()
            test_acc, tr_time, tradeOff_metric = q.get()
            process_eval.join()

            # Print the results.
            print()
            print("========================== EDW EINAI TO CORE.......")
            print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
            print(f"Training time: ", tr_time)
            print(tradeOff_metric)
            print()

            # Delete the dfs.
            del train_images
            del train_labels
            del test_images
            del test_labels


            # Clear the Keras session, otherwise it will keep adding new
            # models to the same TensorFlow graph each time we create
            # a model with a different set of hyper-parameters.
            K.clear_session()
            tf.compat.v1.reset_default_graph()

            # If this is the first combination that is examined then create the dataframe and store the result
            if sample_size == sample_size_low and epochs == num_of_epochs_low and lr == lr_list[0] and size_of_batch == size_of_batch_low:
              df = pd.DataFrame([[['conv'], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
            # else append the result to the existing dataframe
            else:
              df_tmp = pd.DataFrame([[['conv'], sample_size, lr, size_of_batch, epochs, test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
              df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
              del df_tmp

  # If this is not the first step/iteration
  else:
    # This boolean variable indicates if this is the first examined combination in order to create a df
    flag = True

    # Get the df of the previous step/iteration
    df_tmp2 = best_layers[-1]

    # For every pareto optimal solution (of the previous step)
    for pareto_layer in range(0, len(df_tmp2)):
      current_df = df_tmp2.iloc[pareto_layer]

      # For all the available types of layers
      for layer in set_of_layers:

        # After a dense laye you can only add a dense layer
        # So, don't examine the other types of layers
        if current_df['LayerType'][-1] == 'dense' and layer != 'dense':
          continue

        # Perform the sampling
        train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, current_df['SamplingRate'], test_images_all, test_labels_all, perc_test)

        q = Queue()
        process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, layer, current_df, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
        process_eval.start()
        test_acc, tr_time, tradeOff_metric = q.get()
        process_eval.join()

        # Print the results.
        print()
        print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
        print(f"Training time: ", tr_time)
        print(tradeOff_metric)
        print()

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        K.clear_session()
        gc.collect()
        del train_images
        del train_labels
        del test_images
        del test_labels
        #tf.compat.v1.reset_default_graph()

        # If this is the first combination that is examined then create the dataframe and store the result
        if flag:
          flag = False
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
        # else append the result to the existing df
        else:
          so_far_layers = current_df['LayerType'].copy()
          so_far_layers.append(layer)
          df_tmp = pd.DataFrame([[so_far_layers, current_df['SamplingRate'], current_df['LearningRate'], current_df['BatchSize'], current_df['Epochs'], test_acc, tr_time, tradeOff_metric]], columns=['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Accuracy', 'TrainingSpeed', 'Score'])
          df = pd.concat([df, df_tmp], axis = 0, ignore_index=True)
          del df_tmp


  print(df)

  # Keep the columns of the dataframe that we need to compute the Pareto opt, i.e., accuracy and training speed
  df2 = df.loc[:, df.columns.drop(['LayerType', 'SamplingRate', 'LearningRate', 'BatchSize', 'Epochs', 'Score'])]

  # Set the appropriate mask, i.e., max the accuracy, minimize the training speed
  mask = paretoset(df2, sense=["max", "min"])

  # Apply the mask and get the result
  paretoset_nn = df[mask]
  print(paretoset_nn)

  del df2

  # Add the dataframe that contains the pareto opt architectures for this step/iteration and the corresponding metrics
  best_layers.append(paretoset_nn)

Percentage of filtering in our training dataset was set:
0.05
Class 0 has 250 samples in our dataset...
Class 1 has 250 samples in our dataset...
Class 2 has 250 samples in our dataset...
Class 3 has 250 samples in our dataset...
Class 4 has 250 samples in our dataset...
Class 5 has 250 samples in our dataset...
Class 6 has 250 samples in our dataset...
Class 7 has 250 samples in our dataset...
Class 8 has 250 samples in our dataset...
Class 9 has 250 samples in our dataset...
Training dataset before sampling:
(50000, 32, 32, 3)
(50000, 1)
Training dataset after sampling:
(2500, 32, 32, 3)
(2500, 1)
Testing dataset before sampling:
(10000, 32, 32, 3)
(10000, 1)
Testing dataset after sampling:
(10000, 32, 32, 3)
(10000, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 28800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │       288,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 288,906 (1.10 MB)

...

 Trainable params: 1,412,874 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.3725 - loss: 1.7045
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6061 - loss: 1.1287
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6794 - loss: 0.9208
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7232 - loss: 0.7950
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7644 - loss: 0.6719
313/313 - 4s - 12ms/step - accuracy: 0.6892 - loss: 0.9295
========================== EDW EINAI TO MULTI2222222222.......
Accuracy (on the testing dataset): 68.92%
Training time:  14.818288326263428
0.5853074814574604

Accuracy (on the testing dataset): 68.92%
Training time:  14.818288326263428
0.5853074814574604

                           LayerType  SamplingRate  LearningRate  BatchSize  \
0     [conv, conv, pool, conv, conv]          0.35         0.001         64   
1     [conv, conv, pool, conv, pool]          0.35         0.001         64   
2    [conv, conv, pool, conv, de

In [14]:
stop_program = time.time()

# Results

In [15]:
best_layers[-1]

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
1,"[conv, conv, pool, conv, pool]",0.35,0.001,64,5,0.6639,11.984894,0.582234
3,"[conv, conv, pool, pool, conv]",0.35,0.001,64,5,0.6486,11.895403,0.580372
8,"[conv, conv, pool, pool, pool]",0.80,0.001,64,1,0.5525,6.577648,0.526259
9,"[conv, conv, pool, pool, dense]",0.80,0.001,64,1,0.6083,7.350181,0.542810
15,"[conv, pool, conv, pool, pool]",0.80,0.001,64,1,0.4994,6.458289,0.518146
19,"[conv, pool, pool, conv, pool]",0.80,0.001,64,1,0.4783,6.337133,0.513998
21,"[conv, pool, pool, pool, conv]",0.80,0.001,64,1,0.4850,6.338709,0.514854
22,"[conv, pool, pool, pool, pool]",0.80,0.001,64,1,0.3445,5.521791,0.487088
23,"[conv, pool, pool, pool, dense]",0.80,0.001,64,1,0.4690,6.335480,0.512816
25,"[conv, pool, conv, conv, conv]",0.80,0.001,64,5,0.7134,15.887993,0.588241


In [16]:
df_best = best_layers[-1].query('Score == Score.max()')
df_best

,LayerType,SamplingRate,LearningRate,BatchSize,Epochs,Accuracy,TrainingSpeed,Score
25,"[conv, pool, conv, conv, conv]",0.8,0.001,64,5,0.7134,15.887993,0.588241


In [17]:
heuristic_solution = df_best['LayerType'].tolist()
heuristic_solution[0]

['conv', 'pool', 'conv', 'conv', 'conv']

In [18]:
if heuristic_solution[0][-1] != 'dense':
  heuristic_solution[0].append('flatten')
else:
  heuristic_solution[0].insert(heuristic_solution[0].index("dense"), "flatten")
heuristic_solution[0].append('dense_classes_softmax')

In [19]:
print(f"Based on the heuristic approach the best nework is ------> {heuristic_solution[0]}")

Based on the heuristic approach the best nework is ------> ['conv', 'pool', 'conv', 'conv', 'conv', 'flatten', 'dense_classes_softmax']


In [20]:
# Compute the time
program_time = stop_program - start_program
program_time

1550.461894750595